### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{'_wandb': {'runtime': 1}},"{'exp_name': 'dt', 'overrides': [], 'config_di...",2t00r1t2
1,{},"{'env': {'game': 'Alien', 'type': 'atari', 'fr...",28p08j7w
2,{},"{'env': {'game': 'Asterix', 'type': 'atari', '...",jm4j6r4i
3,{},"{'env': {'game': 'Amidar', 'type': 'atari', 'f...",bt9lmdo1
4,{},"{'env': {'game': 'Assault', 'type': 'atari', '...",3jaas2va
...,...,...,...
1250,"{'positive_sim': 0.9272139549255372, 'loss': 0...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1251,"{'_timestamp': 1659280381, 'negative_sim': 0.5...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1252,"{'loss': -0.9783781695365906, '_step': 117001,...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1253,"{'positive_sim': 0.9873969799280168, 'loss': -...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'dt'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_111118/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
5,"{'mean_grad_norm': 0.1147770823501959, 'loss':...","{'env': {'game': 'UpNDown', 'type': 'atari', '...",2ramkg99,UpNDown
6,"{'_runtime': 24686, 'reward_f1': 0.62296269467...","{'env': {'game': 'Seaquest', 'type': 'atari', ...",lagcleg6,Seaquest
11,"{'reward_ratio': 0.07465373268663433, 'best_me...","{'env': {'game': 'RoadRunner', 'type': 'atari'...",2hm5ulzu,RoadRunner
12,"{'epoch': 10, 'act_acc': 0.8544497170099398, '...","{'env': {'game': 'Qbert', 'type': 'atari', 'fr...",o4zhim2r,Qbert
13,"{'act_f1': 0.11289612424892984, 'best_metric_v...","{'env': {'game': 'PrivateEye', 'type': 'atari'...",24y02njd,PrivateEye
14,"{'act_acc': 0.758032405925637, 'reward_f1': 0....","{'env': {'game': 'Pong', 'type': 'atari', 'fra...",2qe8njld,Pong
17,"{'epoch': 10, '_wandb': {'runtime': 24744}, 'a...","{'env': {'game': 'Krull', 'type': 'atari', 'fr...",32k9h8oh,Krull
18,"{'_step': 234370, 'min_grad_norm': 4.104905675...","{'env': {'game': 'KungFuMaster', 'type': 'atar...",5a8xmob9,KungFuMaster
19,"{'_step': 234370, '_wandb': {'runtime': 24801}...","{'env': {'game': 'Kangaroo', 'type': 'atari', ...",2tp6abwu,Kangaroo
20,"{'_step': 234370, '_timestamp': 1668562943, 'm...","{'env': {'game': 'MsPacman', 'type': 'atari', ...",3ufht270,MsPacman


### 1. Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

UpNDown 2ramkg99
Seaquest lagcleg6
RoadRunner 2hm5ulzu
Qbert o4zhim2r
PrivateEye 24y02njd
Pong 2qe8njld
Krull 32k9h8oh
KungFuMaster 5a8xmob9
Kangaroo 2tp6abwu
MsPacman 3ufht270
Gopher 3vgpa2sr
Hero 16n7kdgq
Jamesbond 3r49olri
Frostbite 1fw6yh3w
CrazyClimber 174tmddi
Freeway 1abugb7l
DemonAttack 2v8uuxep
ChopperCommand 24q1q34c
Boxing 18vfabwn
BankHeist 1zf4xle4
BattleZone 177cijhm
Breakout 1y3wa68a
Asterix 2turwfj3
Assault 12vj9kk8
Amidar 1hprwh5d
Alien 25nyxscf


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)